<a href="https://colab.research.google.com/github/vivaming/PyPractice/blob/master/DL_With_Python_Ch05_Image_Classification_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
# !pwd

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content'

In [ ]:
!kaggle competitions download -c dogs-vs-cats
!unzip /content/dogs-vs-cats.zip  -d /content/dogs_cats
!unzip /content/dogs_cats/test1.zip -d /content/dogs_cats
!unzip /content/dogs_cats/train.zip -d /content/dogs_cats

In [ ]:
import os, shutil

set up the original directory and sample directory

In [ ]:
original_data_dir='/content/dogs_cats'
base_dir='/content/dogs_cats_sample'
os.mkdir(base_dir)

In [ ]:
for i in ['train', 'validation', 'test']:
    new_dir=os.path.join(base_dir, i)
    os.mkdir(new_dir)
    for n in ['cats', 'dogs']:
      new_sub_dir=os.path.join(new_dir, n)
      os.mkdir(new_sub_dir)


In [ ]:
train_dir='/content/dogs_cats_sample/train'
validation_dir='/content/dogs_cats_sample/validation'


In [ ]:
file_names=['cat.{}.jpg'.format(i) for i in range(1000)]
for file_name in file_names:
    src=os.path.join(original_data_dir, 'train', file_name)
    dst='/content/dogs_cats_sample/train/cats'
    shutil.copy(src, dst)

In [ ]:
file_names=['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for file_name in file_names:
    src=os.path.join(original_data_dir, 'train', file_name)
    dst='/content/dogs_cats_sample/validation/cats'
    shutil.copy(src, dst)

In [ ]:
file_names=['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for file_name in file_names:
    src=os.path.join(original_data_dir, 'train', file_name)
    dst='/content/dogs_cats_sample/test/cats'
    shutil.copy(src, dst)

In [ ]:
file_names=['dog.{}.jpg'.format(i) for i in range(1000)]
for file_name in file_names:
    src=os.path.join(original_data_dir, 'train', file_name)
    dst='/content/dogs_cats_sample/train/dogs'
    shutil.copy(src, dst)

In [ ]:
file_names=['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for file_name in file_names:
    src=os.path.join(original_data_dir, 'train', file_name)
    dst='/content/dogs_cats_sample/validation/dogs'
    shutil.copy(src, dst)

In [ ]:
file_names=['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for file_name in file_names:
    src=os.path.join(original_data_dir, 'train', file_name)
    dst='/content/dogs_cats_sample/test/dogs'
    shutil.copy(src, dst)

In [ ]:
len(os.listdir('/content/dogs_cats_sample/train/cats'))

Data Preprocessing

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255
                                 , rotation_range=40
                                 , width_shift_range=0.2
                                 , height_shift_range=0.2
                                 , shear_range=0.2
                                 , zoom_range=0.2
                                 , horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=train_datagen.flow_from_directory(
    train_dir
    , target_size=(150, 150)
    , batch_size=20
    , class_mode='binary'
)

validation_generator=test_datagen.flow_from_directory(
    validation_dir
    , target_size=(150, 150)
    , batch_size=20
    , class_mode='binary'
)

In [ ]:
for data_batch, labels_batch in train_generator:
    print('data batch shape: ',  data_batch.shape)
    print('label batch shape: ', labels_batch.shape)
    print("batch: ", labels_batch)
    break

Define the model

In [ ]:
from keras import layers
from keras import models

In [ ]:
# model = models.sequential(
#   [
#    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
#    layers.MaxPooling2D((2, 2)),
#    layers.Conv2D(64, (3, 3), activation='relu'),
#    layers.MaxPooling2D((2, 2)),
#    layers.Conv2D(128, (3, 3), activation='relu'),
#    layers.MaxPooling2D((2, 2)),
#    layers.Conv2D(128, (3, 3), activation='relu'),
#    layers.MaxPooling2D((2, 2)),
#    layers.Flatten(),
#    layers.Dense(512, activation='relu'),
#    layers.Dense(1, activation='sigmoid')
   
#   ]
# )

In [ ]:
model=models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy'
              , optimizer='RMSprop'
              , metrics=['accuracy'])

In [ ]:
history=model.fit_generator(train_generator
                            , steps_per_epoch=100
                            , epochs=100
                            , validation_data=validation_generator)

In [ ]:
model.save('cats_and_dogs_2.h5')
files.download("cats_and_dogs_2.h5")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

In [ ]:
epochs=range(1, len(acc)+1)

In [ ]:
plt.plot(epochs, acc, 'bo', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and validation accuracy')
plt.legend()

In [ ]:
plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validaton loss')
plt.legend()

Extracting features using the pretrained convolutional base

In [52]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.applications.vgg16 import VGG16

In [53]:

conv_base = VGG16(weights='imagenet'
                  , include_top=False
                  , input_shape=(150, 150, 3))

In [54]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [55]:
base_dir='/content/dogs_cats_sample'

In [56]:
train_dir=os.path.join(base_dir, 'train')
validation_dir=os.path.join(base_dir, 'validation')
test_dir=os.path.join(base_dir, 'test')

In [ ]:
datagen=ImageDataGenerator(rescale=1./255)
batch_size=20

In [ ]:
def extract_features(directory, sample_count):
    features=np.zeros(shape=(sample_count, 4, 4, 512))
    labels=np.zeros(shape=(sample_count))
    generator=datagen.flow_from_directory(
        directory
        , target_size=(150,150)
        , batch_size = batch_size
        , class_mode='binary'
    )
    i=0

    for inputs_batch, labels_batch in generator:
        features_batch=conv_base.predict(inputs_batch)
        features[i*batch_size:(i+1)*batch_size] = features_batch
        labels[i*batch_size:(i+1)*batch_size] = labels_batch
        i+=1
        if i*batch_size>=sample_count:
          break
    return features, labels

In [ ]:
train_features, train_labels=extract_features(train_dir, 2000)
validation_features, validation_labels=extract_features(validation_dir, 1000)
test_features, test_labels=extract_features(test_dir, 1000)

In [ ]:
train_features.shape

The extracted features are currently of shape (samples, 4, 4, 512). You’ll feed them to a densely connected classifier, so first you must flatten them to (samples, 8192)

In [ ]:
train_features=np.reshape(train_features, (2000, 4*4*512))
validation_features=np.reshape(validation_features, (1000, 4*4*512))
test_features=np.reshape(test_features, (1000, 4*4*512))

In [ ]:
train_features.shape

In [ ]:
from keras import models
from keras import layers
from keras import optimizers

In [ ]:
model=models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4*4*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='RMSProp'
              , loss='binary_crossentropy'
              , metrics=['accuracy'])

In [ ]:
history=model.fit(train_features, train_labels
                  , epochs=30
                  , batch_size=20
                  , validation_data=(validation_features, validation_labels))

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(1, len(acc)+1)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(epochs, acc, 'bo', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label="Validation Accuarcy")

In [ ]:
plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation Loss')

In [ ]:
model.save('cats_and_dogs_3.h5')
files.download("cats_and_dogs_3.h5")

In [71]:
from keras import models
from keras import layers
from tensorflow.keras import optimizers

In [58]:
model=models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [59]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 256)               2097408   
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


Freeze the conv base

In [60]:
len(model.trainable_weights)

30

In [61]:
conv_base.trainable=False

In [62]:
len(model.trainable_weights)

4

In [63]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

In [64]:
train_datagen=ImageDataGenerator(
    rescale=1./255
    , rotation_range=90
    , width_shift_range=0.2
    , height_shift_range=0.2
    , shear_range=0.2
    , zoom_range=0.2
    , horizontal_flip=True
    ,fill_mode='nearest'
)

In [65]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [66]:
train_generator=train_datagen.flow_from_directory(
    train_dir
    , target_size=(150, 150)
    , batch_size=20
    , class_mode='binary'    
)

Found 2000 images belonging to 2 classes.


In [67]:
validation_generator=test_datagen.flow_from_directory(
    validation_dir
    , target_size=(150, 150)
    , batch_size=20
    , class_mode='binary'
)

Found 1000 images belonging to 2 classes.


In [79]:
model.compile(loss='binary_crossentropy'
              #, optimizer='RMSprop'
              #, optimizer=optimizers.RMSprop(learning_rate=0.01)
              , optimizer='adam'
              , metrics=['Accuracy'])

In [ ]:
history=model.fit_generator(train_generator
          ### number of training batches = uniqueTrainingData / batchSize
          , steps_per_epoch=100
          , epochs=30
          , validation_data=validation_generator
          ### validation_steps = total_validation_samples // validation_batch_size
          , validation_steps=50
          )

Epoch 1/30


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


100/100 [==============================] - 21s 201ms/step - loss: 0.3250 - Accuracy: 0.8600 - val_loss: 0.2698 - val_Accuracy: 0.8910
Epoch 2/30
100/100 [==============================] - 20s 197ms/step - loss: 0.3268 - Accuracy: 0.8560 - val_loss: 0.2695 - val_Accuracy: 0.8920
Epoch 3/30
100/100 [==============================] - 22s 222ms/step - loss: 0.3433 - Accuracy: 0.8425 - val_loss: 0.2622 - val_Accuracy: 0.8940
Epoch 4/30
100/100 [==============================] - 20s 198ms/step - loss: 0.3377 - Accuracy: 0.8610 - val_loss: 0.2725 - val_Accuracy: 0.8880
Epoch 5/30
100/100 [==============================] - 19s 195ms/step - loss: 0.3409 - Accuracy: 0.8465 - val_loss: 0.2573 - val_Accuracy: 0.8970
Epoch 6/30
100/100 [==============================] - 20s 201ms/step - loss: 0.3322 - Accuracy: 0.8545 - val_loss: 0.2595 - val_Accuracy: 0.8950
Epoch 7/30
100/100 [==============================] - 19s 191ms/step - loss: 0.3294 - Accuracy: 0.8495 - val_loss: 0.2616 - val_Accuracy: 0.8

In [ ]:
model.save('cats_and_dogs_4.h5')
files.download("cats_and_dogs_4.h5")

In [ ]:
acc=history.history['Accuracy']
val_acc=history.history['val_Accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(1, len(acc)+1)

In [ ]:
#history.history.keys()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(epochs, acc, 'bo', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label="Validation Accuarcy")
plt.legend()

In [ ]:
plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation Loss')
plt.legend()

Evaluate the model on the test data

In [76]:
test_generator=test_datagen.flow_from_directory(
    test_dir
    , target_size=(150, 150)
    , batch_size=20
    , class_mode='binary'
)

Found 1000 images belonging to 2 classes.


In [77]:
test_loss, test_acc=model.evaluate(test_generator, steps=50)

50/50 [==============================] - 4s 74ms/step - loss: 0.3882 - Accuracy: 0.8590


In [78]:
print("test accuaracy: ", test_acc)

test accuaracy:  0.859000027179718
